# Look at WG met file (subset of WG L2 data file)



## Working toward estimation of bulk fluxes

Will need:  
- lat/lon (1 Hz and 20 Hz)
- SMP21/SGR4 data
- wind (u,v,w)
- all WXT variables
- SST, SSS, sensor depth ("UCTD")

1 Hz variables I need:
- time_1Hz
- latitude_1Hz
- longitude_1Hz
- UCTD_sea_water_temperature
- UCTD_salinity
- UCTD_density
- UCTD_depth
- WXT_rainfall_rate
- WXT_atmospheric_pressure
- WXT_air_temperature
- WXT_relative_humidity
- WXT_wind_speed
- WXT_wind_direction
- SMP21_shortwave_flux
- SGR4_longwave_flux
- _also need some estimate of surface current_

20 Hz variables I need:
- time_20Hz
- latitude_20Hz
- longitude_20Hz
- wind_speed
- wind_direction
- wind_vertical

Not sure if I need:  
- time_15min
- wave_frequency


In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib
import datetime as dt
################
# This allows us to import Tom_tools
import sys
sys.path.append('../../Tom_tools/') # you may need to adjust this path
# sys.path.append('../SWOT_IW_SSH/jtf/Tom_tools/') # you may need to adjust this path
import Tom_tools_v1 as tt
################

In [2]:
# %matplotlib inline
%matplotlib qt 
savefig = False # set to true to save plots as file
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 400
plt.close('all')
plt.rcParams['axes.xmargin'] = 0


__figdir__ = '../plots/WG_timeseries/'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
# path = '/mnt/e/SMODE_data/pilot/WG/L2/'
path = '/mnt/c/D_drive/SMODE_data/pilot/WG/L2/'
WG = 'WHOI43'
file = 'SMODE_PFC_Wavegliders_'+WG+'.nc'

In [4]:
naninterp='True' # Interpolate NaNs

In [5]:
# This needs to be set before any dates are encoded as datetime64
# Because the time increment is so small, we get 'out of range' when starting from the standard datetime64 epoch of 1970
# matplotlib.dates.set_epoch('2000-01-01T00:00:00') 

In [6]:
%%time
ds = xr.open_dataset(path+file, engine = 'netcdf4', decode_times = True) #decode_times = False, 

CPU times: user 633 ms, sys: 314 ms, total: 947 ms
Wall time: 1.48 s


In [7]:
ds

<xarray.Dataset>
Dimensions:                     (time_1Hz: 771961, time_10Hz: 7719601, time_15min: 858, wave_frequency: 153, time_20Hz: 15439201, Workhorse_time: 771960, Workhorse_range: 50)
Coordinates: (12/16)
  * time_1Hz                    (time_1Hz) datetime64[ns] 2021-10-28T00:59:59...
    latitude_1Hz                (time_1Hz) float64 ...
    longitude_1Hz               (time_1Hz) float64 ...
  * time_10Hz                   (time_10Hz) datetime64[ns] 2021-10-28T00:59:5...
    altitude_10Hz               (time_10Hz) float32 ...
  * time_15min                  (time_15min) datetime64[ns] 2021-10-28T00:59:...
    ...                          ...
    altitude_20Hz               (time_20Hz) float32 ...
  * Workhorse_time              (Workhorse_time) datetime64[ns] 2021-10-28T01...
  * Workhorse_range             (Workhorse_range) float32 49.0 nan ... nan nan
    Workhorse_longitude         (Workhorse_time) float64 ...
    Workhorse_latitude          (Workhorse_time) float64 ...
    Workhorse_altitude          (Workhorse_time) float64 ...
Data variables: (12/50)
    INST_RBR_Concerto           float64 ...
    INST_Gill                   float64 ...
    INST_SMP21                  float64 ...
    INST_SGR4                   float64 ...
    INST_upper_ctd              float64 ...
    INST_WXT                    float64 ...
    ...                          ...
    Workhorse_heading           (Workhorse_time) float32 ...
    Workhorse_temperature       (Workhorse_time) float32 ...
    Workhorse_vel_east          (Workhorse_time, Workhorse_range) float32 ...
    Workhorse_vel_north         (Workhorse_time, Workhorse_range) float32 ...
    Workhorse_vel_up            (Workhorse_time, Workhorse_range) float32 ...
    Workhorse_flag              (Workhorse_time, Workhorse_range) float32 ...
Attributes: (12/40)
    DOI:                           10.5067/SMODE-GLID3
    acknowledgement:               This work is a contribution to the S-MODE ...
    keywords_vocabulary:           NASA Global Change Master Directory (GCMD)...
    Conventions:                   CF-1.8, ACDD-1.3
    id:                            PO.DAAC-SMODE-GLID3
    naming_authority:              gov.nasa
    ...                            ...
    summary:                       Wave Glider WHOI43 was deployed by R/V Oce...
    uuid:                          5aa1a549-1112-4ae7-aa8f-92f2a8f992e8
    history:                       2022-06-09 Benjamin Greenwood - process in...
    date_created:                  2022-06-09T16:02:37Z
    keywords:                      EARTH SCIENCE > OCEANS > SALINITY/DENSITY ...
    instruments:                   In Situ/Laboratory Instruments > Recorders...

In [8]:
# Raw met plot from WG:
fig, axs = plt.subplots(5, 1, sharex=True)
fig.autofmt_xdate()
plt.subplot(5,1,1)
h1, = plt.plot(ds.time_1Hz, ds.WXT_air_temperature)
h2, = plt.plot(ds.time_1Hz, ds.UCTD_sea_water_temperature)
plt.legend([h1, h2],['Air temp.','SST'])
plt.ylabel('T [$^\circ$C]')
plt.title('Raw 1 Hz WXT measurements')

plt.subplot(5,1,2)
plt.plot(ds.time_1Hz, ds.WXT_relative_humidity)
plt.ylabel('[%]')
plt.legend(['Rel. Humidity'])

plt.subplot(5,1,3)
plt.plot(ds.time_15min, ds.wave_significant_height)
plt.ylabel('[m]')
plt.legend(['Sig. wave height'],loc='upper right')

plt.subplot(5,1,4)
plt.plot(ds.time_20Hz, ds.wind_speed)
plt.plot(ds.time_1Hz, ds.WXT_wind_speed)
plt.ylabel('[m/s]')
plt.legend(['Gill Wind speed','WXT wind speed'],loc='upper right')

plt.subplot(5,1,5)
plt.plot(ds.Workhorse_time, ds.Workhorse_altitude)
plt.ylabel('[m]')
plt.legend(['Workhorse altitude (from IMU/GPS)'],loc='upper right')

if savefig:
    plt.savefig(__figdir__+WG+'_raw_met' + '.' +plotfiletype,**savefig_args)


In [9]:
# Raw met plot from WG:
fig, axs = plt.subplots(4, 1, sharex=True)
fig.autofmt_xdate()
plt.subplot(4,1,1)
plt.plot(ds.time_1Hz, ds.WXT_atmospheric_pressure)
plt.ylabel('[mbar]')
plt.legend(['Atm. pressure'])

plt.subplot(4,1,2)
plt.plot(ds.time_1Hz, ds.SGR4_longwave_flux)
plt.ylabel('W/m^2')
plt.legend(['Longwave radiation'])

plt.subplot(4,1,3)
plt.plot(ds.time_1Hz, ds.SMP21_shortwave_flux)
plt.ylabel('W/m^2')
plt.legend(['Shortwave radiation'])

plt.subplot(4,1,4)
plt.plot(ds.time_1Hz, ds.WXT_rainfall_rate)
plt.ylabel('[mm/hr]')
plt.legend(['Precip. rate'])

if savefig:
    plt.savefig(__figdir__+WG+'_raw_met2' + '.' +plotfiletype,**savefig_args)


In [10]:
# Raw met plot from WG:
fig, axs = plt.subplots(5, 1, sharex=True,figsize=(5,7))
fig.autofmt_xdate()
plt.subplot(5,1,1)
h1, = plt.plot(ds.time_1Hz, ds.WXT_air_temperature)
h2, = plt.plot(ds.time_1Hz, ds.UCTD_sea_water_temperature)
plt.legend([h1, h2],['Air temp.','SST'])
plt.ylabel('T [$^\circ$C]')
plt.title('Wave Glider surface measurements')

plt.subplot(5,1,2)
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.SGR4_longwave_flux,15*60))
plt.ylabel('W/m^2')
plt.legend(['Longwave radiation'])
plt.setp(plt.gca().get_xticklabels(), visible=False)

plt.subplot(5,1,3)
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.SMP21_shortwave_flux,15*60))
plt.ylabel('W/m^2')
plt.legend(['Shortwave radiation'])

plt.subplot(5,1,4)
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.WXT_wind_speed,15*60))
plt.ylabel('[m/s]')
plt.legend(['wind speed'],loc='upper right')

plt.subplot(5,1,5)
plt.plot(ds.time_15min, ds.wave_significant_height)
plt.ylabel('[m]')
plt.legend(['Sig. wave height'],loc='upper right')

if savefig:
    plt.savefig(__figdir__+WG+'_raw_met3' + '.' +plotfiletype,**savefig_args)


In [11]:
# There is a limited number of NaNs in WXT wind speed
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_1Hz, np.isnan(ds.WXT_wind_speed.values))
plt.title('NaNs in WXT wind speed')

Text(0.5, 1.0, 'NaNs in WXT wind speed')

In [12]:
# Let's interpolate them
ff = np.where(np.isnan(ds.WXT_wind_speed.values)==0)
t = ds.time_1Hz[ff]
wspd = ds.WXT_wind_speed[ff]
WXT_wspd_interp = np.interp(ds.time_1Hz, t, wspd)
print(str(np.round(100*(1-np.size(ff)/np.size(ds.time_1Hz)),3))+'% of values are NaN')

0.014% of values are NaN


In [13]:
'''
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

# plt.plot(t, np.isnan(wspd))
plt.plot(np.isnan(WXT_wspd_interp))
plt.title('NaNs in WXT wind speed after trying to remove nans')
''' # this checks out

"\nfig, axs = plt.subplots(1, 1)\nfig.autofmt_xdate()\n\n# plt.plot(t, np.isnan(wspd))\nplt.plot(np.isnan(WXT_wspd_interp))\nplt.title('NaNs in WXT wind speed after trying to remove nans')\n"

In [14]:
# There is also a limited number of NaNs in Gill wind speed
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_20Hz, np.isnan(ds.wind_speed.values))
plt.title('NaNs in Gill wind speed')

Text(0.5, 1.0, 'NaNs in Gill wind speed')

In [15]:
# Let's interpolate them
ff = np.where(np.isnan(ds.wind_speed.values)==0)
t = ds.time_20Hz[ff]
wspd = ds.wind_speed[ff]
wspd_interp = np.interp(ds.time_20Hz, t, wspd)

In [16]:
fig, axs = plt.subplots(1, 1)
M=151
tt.spectrum_band_avg(wspd_interp,1/20,M,winstr=None,plotflag=True,ebarflag=None)
tt.spectrum_band_avg(WXT_wspd_interp,1,M,winstr=None,plotflag=True,ebarflag=False)
plt.title('Gill, WXT wind speed')
plt.xlabel('Hz')
plt.ylabel('Spectral density [m$^2$/s$^2$/Hz]')
plt.legend(['Gill Wind speed','WXT wind speed'],loc='upper right')
if savefig:
    plt.savefig(__figdir__+WG+'_wsp_spectra' + '.' +plotfiletype,**savefig_args)


## There is still a wave signal in the motion corected velocities, but it may be real, and in any case, it shouldn't be a big issue when we average to 1 min or longer

In [17]:
fig, axs = plt.subplots(1, 1, sharex=True)
fig.autofmt_xdate()
plt.plot(ds.time_20Hz, tt.run_avg1d(ds.wind_speed,15*60*20))
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.WXT_wind_speed,15*60))
plt.ylabel('[m/s]')
plt.legend(['Gill Wind speed ('+str(ds.INST_Gill.height_above_waterline_m)+'m)','WXT wind speed ('+str(ds.INST_WXT.height_above_waterline_m)+'m)'],loc='upper right')
plt.title('15-min average Gill R-3 and WXT wind speed')
plt.axis('tight')
if savefig:
    plt.savefig(__figdir__+WG+'_wspd_smoothed' + '.' +plotfiletype,**savefig_args)


In [18]:
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.Workhorse_time, ds.Workhorse_altitude)
plt.title('Workhorse altitude\n Something bad happened with the VectorNav?')
plt.xlim(18935.65223174665, 18936.366246499798)

if savefig:
    plt.savefig(__figdir__+WG+'_Workhorse_altitude' + '.' +plotfiletype,**savefig_args)


In [19]:
xlim = plt.xlim()

In [20]:
xlim

(18935.65223174665, 18936.366246499798)

## Make an averaged data set (say, 1 min)

### One thing to remember is that the time is not necessarily uniformly spaced.
But, for the WHOI WG, time_1Hz is ok:
```np.max(np.diff(ds.time_1Hz))``` is about 1 sec

It would probably be smarter to automate this to loop through all ds.time_1Hz variables (for example).  That might look something like this:  
```
var_list = []  
for var in ds.data_vars.variables:  
    var_list.append(var)  
    print(var)  
```
Or maybe like this:
```
#Copy dimensions
for dname, the_dim in dsin.dimensions.iteritems():
    print dname, len(the_dim)
    dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
```

OK, think about the long view of how I want to do this.  
* One thing that would be nice would be to just cycle through the all of the 1 Hz variables (for example)
* I'd like to take the names of the variables, something like the examples above
* then do the filtering
* then copy the metadata
* assign it to a new xarray dataset, maybe like this, https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html

In [21]:
ds.WXT_wind_direction

<xarray.DataArray 'WXT_wind_direction' (time_1Hz: 771961)>
[771961 values with dtype=float32]
Coordinates:
  * time_1Hz       (time_1Hz) datetime64[ns] 2021-10-28T00:59:59.999996672 .....
    latitude_1Hz   (time_1Hz) float64 ...
    longitude_1Hz  (time_1Hz) float64 ...
Attributes:
    long_name:              Wind direction measured by WXT530; positive towar...
    standard_name:          wind_to_direction
    units:                  degree
    valid_min:              0.0
    valid_max:              360.0
    sensor:                 INST_WXT
    coverage_content_type:  physicalMeasurement

In [22]:
# Make wind vector before smoothing
ds['WXT_U'] = ds.WXT_wind_speed*np.cos(ds.WXT_wind_direction*np.pi/180)
ds['WXT_V'] = ds.WXT_wind_speed*np.sin(ds.WXT_wind_direction*np.pi/180)
ds['wind_east']=ds.wind_speed*np.cos(ds.wind_direction*np.pi/180)
ds['wind_north']=ds.wind_speed*np.sin(ds.wind_direction*np.pi/180)


In [23]:
# This is very close to what I want!
#  next step can use ds.data_vars.get(var) or something

var_list = []  
not_used = []  
for var in ds.data_vars:
    try:
        if ds.data_vars.get(var).dims[0]=='time_1Hz':
            var_list.append(var)
            print(var)
    except:
        not_used.append(var)


UCTD_depth
UCTD_sea_water_temperature
UCTD_conductivity
UCTD_salinity
UCTD_density
WXT_rainfall_rate
WXT_atmospheric_pressure
WXT_air_temperature
WXT_relative_humidity
WXT_wind_speed
WXT_wind_direction
SMP21_shortwave_flux
SGR4_longwave_flux
WXT_U
WXT_V


In [24]:
ds['WXT_U']

<xarray.DataArray 'WXT_U' (time_1Hz: 771961)>
array([ 4.1013265 ,  4.427948  ,  4.483992  , ...,  0.32228282,
        0.29299754, -0.1227663 ], dtype=float32)
Coordinates:
  * time_1Hz       (time_1Hz) datetime64[ns] 2021-10-28T00:59:59.999996672 .....
    latitude_1Hz   (time_1Hz) float64 ...
    longitude_1Hz  (time_1Hz) float64 ...

In [25]:
nsec=60 # number of seconds to average

In [26]:
# Let's interpolate them
ff = np.where(np.isnan(ds.WXT_wind_speed.values)==0)
t = ds.time_1Hz[ff]
wspd = ds.WXT_wind_speed[ff]
WXT_wspd_interp = np.interp(ds.time_1Hz, t, wspd)
print(str(np.round(100*(1-np.size(ff)/np.size(ds.time_1Hz)),3))+'% of values are NaN')

0.014% of values are NaN


In [27]:
naninterp

'True'

In [28]:
ds_new = xr.Dataset().assign_attrs(ds.attrs)  # make empty xr.Dataset but copy attributes from original file
for var in var_list:
    var1Hz = ds.data_vars.get(var)
    if naninterp=='True': # Interpolate NaNs
        ff = np.where(np.isnan(var1Hz)==0)
        t = ds.time_1Hz[ff]
        var_noninterp = var1Hz[ff]
        var_input = np.interp(ds.time_1Hz, t, var_noninterp)
    else: #just provide raw data as input to smoothing
        var_input = var1Hz
    
    var_value = tt.run_avg1d(var_input,nsec) #Smooth over nsec seconds
    locals()[var+'_low'] = var1Hz.rename(var+'_low') #locals()['string'] makes a variable with the name string
    print(var+'_low created, '+str(np.round(100*(1-np.size(ff)/np.size(ds.time_1Hz)),3))+'% of values are NaN')
    locals()[var+'_low'].values = var_value
    ds_new[var+'_low'] = locals()[var+'_low']
    


UCTD_depth_low created, 0.008% of values are NaN
UCTD_sea_water_temperature_low created, 0.0% of values are NaN
UCTD_conductivity_low created, 0.0% of values are NaN
UCTD_salinity_low created, 0.0% of values are NaN
UCTD_density_low created, 0.008% of values are NaN
WXT_rainfall_rate_low created, 0.006% of values are NaN
WXT_atmospheric_pressure_low created, 0.006% of values are NaN
WXT_air_temperature_low created, 0.006% of values are NaN
WXT_relative_humidity_low created, 0.006% of values are NaN
WXT_wind_speed_low created, 0.014% of values are NaN
WXT_wind_direction_low created, 0.014% of values are NaN
SMP21_shortwave_flux_low created, 0.0% of values are NaN
SGR4_longwave_flux_low created, 0.0% of values are NaN
WXT_U_low created, 0.014% of values are NaN
WXT_V_low created, 0.014% of values are NaN


In [29]:
var1Hz

<xarray.DataArray 'WXT_V' (time_1Hz: 771961)>
array([-2.01556129,         nan,         nan, ...,         nan,
               nan, -1.52752024])
Coordinates:
  * time_1Hz       (time_1Hz) datetime64[ns] 2021-10-28T00:59:59.999996672 .....
    latitude_1Hz   (time_1Hz) float64 ...
    longitude_1Hz  (time_1Hz) float64 ...

In [30]:
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds_new.time_1Hz,ds_new.WXT_U_low)
plt.plot(ds_new.time_1Hz,ds_new.WXT_V_low)
plt.plot(ds_new.time_1Hz,ds_new.WXT_wind_speed_low)
plt.title('Averaged WXT wind')
plt.ylabel('m/s')
#plt.ylim(-2,2)
plt.legend(['U','V','speed'])


In [31]:
# Now do 20 Hz variables (IMU and Gill sonic)
var_list = []  
not_used = []  
for var in ds.data_vars:
    try:
        if ds.data_vars.get(var).dims[0]=='time_20Hz':
            var_list.append(var)
            print(var)
    except:
        not_used.append(var)


heading
pitch
roll
u_waveglider
v_waveglider
w_waveglider
Gill_temperature
wind_speed
wind_direction
wind_vertical
wind_east
wind_north


In [32]:
# if making function, need npts=fsamp*nsec as input
for var in var_list:
    foo = ds.data_vars.get(var)
    foo_value = tt.run_avg1d(foo,20*nsec) #Smooth over nsec seconds
    locals()[var+'_low'] = ds.WXT_wind_speed.rename(var+'_low') #copying an arbitrary time_1Hz variable; locals()['string'] makes a variable with the name string
    print(var+'_low created')
    locals()[var+'_low'].values = np.append(foo_value[0:-1:20],np.nan) # this is probably not robust-- for WHOI43 time_20Hz[0:-1:20] is 1 element shorter than time_1Hz
    ds_new[var+'_low'] = locals()[var+'_low'].assign_attrs(ds[var].attrs)


heading_low created
pitch_low created
roll_low created
u_waveglider_low created
v_waveglider_low created
w_waveglider_low created
Gill_temperature_low created
wind_speed_low created
wind_direction_low created
wind_vertical_low created
wind_east_low created
wind_north_low created


In [33]:
# Now do Workhorse variables
var_list = []  
not_used = []  
for var in ds.data_vars:
    try:
        if ds.data_vars.get(var).dims[0]=='Workhorse_time':
            var_list.append(var)
            print(var)
    except:
        not_used.append(var)


Workhorse_roll
Workhorse_pitch
Workhorse_heading
Workhorse_temperature
Workhorse_vel_east
Workhorse_vel_north
Workhorse_vel_up
Workhorse_flag


In [34]:
# if making function, need npts=fsamp*nsec as input
for var in var_list:
    foo = ds.data_vars.get(var)
    try: #if 1d
        foo_value = tt.run_avg1d(foo,nsec) #Smooth over nsec seconds
    except: #if 2d
        foo_value = tt.run_avg2d(foo,nsec,1) #Smooth over nsec seconds        
    locals()[var+'_low'] = foo.rename(var+'_low') #Somehow need to get time_1Hz as coord; locals()['string'] makes a variable with the name string
    print(var+'_low created')
    locals()[var+'_low'].values = foo_value
    ds_new[var+'_low'] = locals()[var+'_low'].assign_attrs(ds[var].attrs)


Workhorse_roll_low created
Workhorse_pitch_low created
Workhorse_heading_low created
Workhorse_temperature_low created
Workhorse_vel_east_low created
Workhorse_vel_north_low created
Workhorse_vel_up_low created
Workhorse_flag_low created


In [35]:
ds_new

<xarray.Dataset>
Dimensions:                         (time_1Hz: 771961, Workhorse_time: 771960, Workhorse_range: 50)
Coordinates:
  * time_1Hz                        (time_1Hz) datetime64[ns] 2021-10-28T00:5...
    latitude_1Hz                    (time_1Hz) float64 ...
    longitude_1Hz                   (time_1Hz) float64 ...
  * Workhorse_time                  (Workhorse_time) datetime64[ns] 2021-10-2...
    Workhorse_longitude             (Workhorse_time) float64 ...
    Workhorse_latitude              (Workhorse_time) float64 ...
    Workhorse_altitude              (Workhorse_time) float64 -3.802 ... -4.864
  * Workhorse_range                 (Workhorse_range) float32 49.0 nan ... nan
Data variables: (12/35)
    UCTD_depth_low                  (time_1Hz) float64 0.2389 nan ... nan 0.2096
    UCTD_sea_water_temperature_low  (time_1Hz) float64 7.159 nan ... nan 7.757
    UCTD_conductivity_low           (time_1Hz) float64 2.039 nan ... nan 2.09
    UCTD_salinity_low               (time_1Hz) float64 16.82 nan ... nan 16.92
    UCTD_density_low                (time_1Hz) float64 512.5 nan ... nan 529.2
    WXT_rainfall_rate_low           (time_1Hz) float64 0.0 nan ... nan 0.0001667
    ...                              ...
    Workhorse_heading_low           (Workhorse_time) float64 -93.96 ... -42.76
    Workhorse_temperature_low       (Workhorse_time) float64 7.322 nan ... 7.816
    Workhorse_vel_east_low          (Workhorse_time, Workhorse_range) float64 ...
    Workhorse_vel_north_low         (Workhorse_time, Workhorse_range) float64 ...
    Workhorse_vel_up_low            (Workhorse_time, Workhorse_range) float64 ...
    Workhorse_flag_low              (Workhorse_time, Workhorse_range) float64 ...
Attributes: (12/40)
    DOI:                           10.5067/SMODE-GLID3
    acknowledgement:               This work is a contribution to the S-MODE ...
    keywords_vocabulary:           NASA Global Change Master Directory (GCMD)...
    Conventions:                   CF-1.8, ACDD-1.3
    id:                            PO.DAAC-SMODE-GLID3
    naming_authority:              gov.nasa
    ...                            ...
    summary:                       Wave Glider WHOI43 was deployed by R/V Oce...
    uuid:                          5aa1a549-1112-4ae7-aa8f-92f2a8f992e8
    history:                       2022-06-09 Benjamin Greenwood - process in...
    date_created:                  2022-06-09T16:02:37Z
    keywords:                      EARTH SCIENCE > OCEANS > SALINITY/DENSITY ...
    instruments:                   In Situ/Laboratory Instruments > Recorders...

In [37]:
ds_new

<xarray.Dataset>
Dimensions:                         (time_1Hz: 771961, Workhorse_time: 771960, Workhorse_range: 50)
Coordinates:
  * time_1Hz                        (time_1Hz) datetime64[ns] 2021-10-28T00:5...
    latitude_1Hz                    (time_1Hz) float64 ...
    longitude_1Hz                   (time_1Hz) float64 ...
  * Workhorse_time                  (Workhorse_time) datetime64[ns] 2021-10-2...
    Workhorse_longitude             (Workhorse_time) float64 ...
    Workhorse_latitude              (Workhorse_time) float64 ...
    Workhorse_altitude              (Workhorse_time) float64 -3.802 ... -4.864
  * Workhorse_range                 (Workhorse_range) float32 49.0 nan ... nan
Data variables: (12/35)
    UCTD_depth_low                  (time_1Hz) float64 0.2389 nan ... nan 0.2096
    UCTD_sea_water_temperature_low  (time_1Hz) float64 7.159 nan ... nan 7.757
    UCTD_conductivity_low           (time_1Hz) float64 2.039 nan ... nan 2.09
    UCTD_salinity_low               (time_1Hz) float64 16.82 nan ... nan 16.92
    UCTD_density_low                (time_1Hz) float64 512.5 nan ... nan 529.2
    WXT_rainfall_rate_low           (time_1Hz) float64 0.0 nan ... nan 0.0001667
    ...                              ...
    Workhorse_heading_low           (Workhorse_time) float64 -93.96 ... -42.76
    Workhorse_temperature_low       (Workhorse_time) float64 7.322 nan ... 7.816
    Workhorse_vel_east_low          (Workhorse_time, Workhorse_range) float64 ...
    Workhorse_vel_north_low         (Workhorse_time, Workhorse_range) float64 ...
    Workhorse_vel_up_low            (Workhorse_time, Workhorse_range) float64 ...
    Workhorse_flag_low              (Workhorse_time, Workhorse_range) float64 ...
Attributes: (12/40)
    DOI:                           10.5067/SMODE-GLID3
    acknowledgement:               This work is a contribution to the S-MODE ...
    keywords_vocabulary:           NASA Global Change Master Directory (GCMD)...
    Conventions:                   CF-1.8, ACDD-1.3
    id:                            PO.DAAC-SMODE-GLID3
    naming_authority:              gov.nasa
    ...                            ...
    summary:                       Wave Glider WHOI43 was deployed by R/V Oce...
    uuid:                          5aa1a549-1112-4ae7-aa8f-92f2a8f992e8
    history:                       2022-06-09 Benjamin Greenwood - process in...
    date_created:                  2022-06-09T16:02:37Z
    keywords:                      EARTH SCIENCE > OCEANS > SALINITY/DENSITY ...
    instruments:                   In Situ/Laboratory Instruments > Recorders...

In [38]:
# Write averaged data to netcdf file
new_file = path+WG+'_L2b_met.nc'
print ('saving to ', new_file)
ds_new.to_netcdf(path=new_file)
ds_new.close()
print ('finished saving')

saving to  /mnt/c/D_drive/SMODE_data/pilot/WG/L2/WHOI43_L2b_met.nc
finished saving


In [ ]:
skfhdskahfka

In [ ]:
# compare different time vectors to decide how to line up variables from different time bases
# ds.time_20Hz
# ds.time_1Hz
# ds.Workhorse_time

# Try the simple thing
# xy, x_ind, y_ind = np.intersect1d(ds.time_1Hz, ds.time_20Hz, assume_unique=True, return_indices=True)
# That doesn't work (no surprise) because the times don't line up exactly (possibly numerical precision issue)

# I think they are already nearly lined up
ds.time_1Hz[0:10:1]-ds.time_20Hz[0:200:20]
# so, try 0,20,40... in 20 Hz array

tfoo = ds.time_20Hz[0:-1:20]
#tfoo and time_1Hz are very close, within 10^-4 sec



In [ ]:
# ds.time_20Hz variables
# wdir is positive clockwise from North
# need to make U, V
Gill_U=ds.wind_speed*np.cos(ds.wind_direction*np.pi/180)
Gill_V=ds.wind_speed*np.sin(ds.wind_direction*np.pi/180)
Gill_wspd_low = tt.run_avg1d(ds.wind_speed, nsec*20)
Gill_U_low = tt.run_avg1d(Gill_U,nsec*20)
Gill_V_low = tt.run_avg1d(Gill_V,nsec*20)
pitch_low = tt.run_avg1d(ds.pitch,nsec*20)
roll_low = tt.run_avg1d(ds['roll'],nsec*20)

In [ ]:
# ds.time_1Hz variables
WXT_U=ds.WXT_wind_speed*np.cos(ds.WXT_wind_direction/np.pi/2)
WXT_V=ds.WXT_wind_speed*np.sin(ds.WXT_wind_direction/np.pi/2)
WXT_U_low = tt.run_avg1d(WXT_U,nsec)
WXT_V_low = tt.run_avg1d(WXT_V,nsec)
WXT_wspd_low = tt.run_avg1d(ds.WXT_wind_speed,nsec)
WXT_atmp_low = tt.run_avg1d(ds.WXT_air_temperature,nsec)
WXT_rh_low = tt.run_avg1d(ds.WXT_relative_humidity,nsec)
swr_low = tt.run_avg1d(ds.SMP21_shortwave_flux,nsec)
lwr_low = tt.run_avg1d(ds.SGR4_longwave_flux,nsec)
lat_low = tt.run_avg1d(ds.latitude_1Hz,nsec)
lon_low = tt.run_avg1d(ds.longitude_1Hz,nsec)

In [ ]:
# Workhorse_time variables
U_low=tt.run_avg2d(ds.Workhorse_vel_east,nsec,1)
V_low=tt.run_avg2d(ds.Workhorse_vel_north,nsec,1)
W_low=tt.run_avg2d(ds.Workhorse_vel_up,nsec,1)

In [ ]:
# Subsample smoothed 20 Hz vars to 1 Hz
Gill_wspd_low = Gill_wspd_low[0:-1:20]
Gill_U_low = Gill_U_low[0:-1:20]
Gill_V_low = Gill_V_low[0:-1:20]
pitch_low = pitch_low[0:-1:20]
roll_low = roll_low[0:-1:20]


In [ ]:
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_1Hz[0:-1:],pitch_low)
plt.plot(ds.time_1Hz[0:-1:],roll_low)
plt.title('1-hr average pitch and roll')
plt.ylabel('Degrees')
plt.ylim(-2,2)

if savefig:
    plt.savefig(__figdir__+WG+'_mean_pitch_roll' + '.' +plotfiletype,**savefig_args)


In [ ]:
plt.ylim(-2,2)

In [ ]:
new_file = path+WG+'_L2b_met.nc'
print ('saving to ', new_file)
ds_new.to_netcdf(path=new_file)
ds_new.close()
print ('finished saving')

In [ ]:
WG

/home/jtomf/miniconda3/lib/python3.9/site-packages/ipykernel/eventloops.py:105: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  app.exec_()


## Preparing for flux computation

Inputs for COARE 3.5:  
    u = ocean surface relative wind speed (m/s) at height zu(m)  
    t = bulk air temperature (degC) at height zt(m)  
    rh = relative humidity (%) at height zq(m)  
    ts = sea water temperature (degC) - see jcool below  
    P = surface air pressure (mb) (default = 1015)  
    Rs = downward shortwave radiation (W/m^2) (default = 150)  
    Rl = downward longwave radiation (W/m^2) (default = 370)  
    zu = wind sensor height (m) (default = 18m)  
    zt = bulk temperature sensor height (m) (default = 18m)  
    zq = RH sensor height (m) (default = 18m)  
    lat = latitude (default = 45 N)  
    zi = PBL height (m) (default = 600m)  
    rain = rain rate (mm/hr)  
    cp = phase speed of dominant waves (m/s)  
    sigH =  significant wave height (m)  
    jcool = cool skin option (default = 1 for bulk SST)  
    
Note: I don't see an input for SST depth--> That's because there is no Warm Layer correction in this version of the code.  

Here is a link to an older matlab version that does have it:  
https://github.com/carsonwitte/Falkor-DWL-Code  

It wouldn't be that hard to add the WL correction-- basically, estimate fluxes, estimate WL correction, and re-estimate fluxes.  


  
  
Required inputs:  
u	zu	t	zt	rh	zq	P	ts	Rs	Rl	lat	zi	rain	cp	sigH


This could be a good way to save the output, but maybe not:  

```
A = coare35vn(u, ta, rh, ts, P=Pa, Rs=rs, Rl=rl, zu=16, zt=16, zq=16,
                lat=Lat, zi=ZI, rain=Rain, jcool=1)
fnameA = os.path.join(path,'test_35_output_py_04022022.txt')
A_hdr = 'usr\ttau\thsb\thlb\thlwebb\ttsr\tqsr\tzot\tzoq\tCd\t'
A_hdr += 'Ch\tCe\tL\tzet\tdter\tdqer\ttkt\tRF\tCdn_10\tChn_10\tCen_10'
np.savetxt(fnameA,A,fmt='%.18e',delimiter='\t',header=A_hdr)
```
